In [176]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import re
import pickle

In [177]:
with open("embedding_matrix.pkl", "rb") as f:
    embedding_matrix = pickle.load(f) 

In [178]:
import re 

replace_puncts = {'`': "'", '′': "'", '“':'"', '”': '"', '‘': "'"}

strip_chars = [',', '.', '"', ':', ')', '(', '-', '|', ';', "'", '[', ']', '>', '=', '+', '\\', '•',  '~', '@', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']

puncts = ['!', '?', '$', '&', '/', '%', '#', '*','£']

def clean_str(x):
    x = str(x)
    
    x = x.lower()
    
    x = re.sub(r"(https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9][a-zA-Z0-9-]+[a-zA-Z0-9]\.[^\s]{2,}|https?:\/\/(?:www\.|(?!www))[a-zA-Z0-9]\.[^\s]{2,}|www\.[a-zA-Z0-9]\.[^\s]{2,})", "url", x)
    
    for k, v in replace_puncts.items():
        x = x.replace(k, f' {v} ')
        
    for punct in strip_chars:
        x = x.replace(punct, ' ') 
    
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
        
    x = x.replace(" '", " ")
    x = x.replace("' ", " ")
        
    return x

In [268]:
from tensorflow.keras.layers import *
from tensorflow.keras.initializers import Constant
model = tf.keras.models.Sequential()

model.add(Embedding(num_words,
                    embedding_dim,
                    embeddings_initializer=Constant(embedding_matrix),
                    input_length=sequence_length))
model.add(SpatialDropout1D(0.2))

model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))

model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.25))
model.add(Dense(units=4, activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])

In [188]:
model.load_weights("model1.h5")

In [224]:

def process_reviews(reviews):
    for i,j in enumerate(reviews):
        reviews[i] = clean_str(j)
    sequence_length = 300
    max_features = 20000
    tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=max_features, split=' ', oov_token='<unw>', filters=' ')
    tokenizer.fit_on_texts(reviews)
    X = tokenizer.texts_to_sequences(reviews)
    X = tf.keras.preprocessing.sequence.pad_sequences(X, sequence_length)
    return X
        

In [258]:
X = process_reviews(["""After watching a ton of his shows on TV, I've been wanting to try some of his signature items to see if he lives up to his reputation.

This is one of his Vegas flagships. It was my first time here but hubby had been here before and loved it. Trendy decor and cute story about going through the Chunnel from Paris to London when you're being seated.

Excellent service at the table! The hostesses aren't the most consistent, but the table service is excellent. Nice explanation of the menu and steak cuts.

These are the highlights and what I would definitely recommend getting:
- Beef Wellington - it's the signature item and sooo good. The layers are perfect and the meat inside is incredibly flavorful
- Sticky toffee pudding - another classic dish that totally lived up to the hype. Definitely good for sharing after a big meal.

Bread service is lovely with a nice variety to choose from, and we also enjoyed the salad to start (not unique, but a nice starter if you need vegetables).  The Mac and cheese side is totall…"""
                     , "Great restaurant with great food, so great"
                ])

In [265]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
X

Found 72 unique tokens.


array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,  14,  28,   6,  29,
          7,   8,  30,  31,  32,  15,  33,  16,  34,   4,  35,  36,   7,
          8,  17,  37,   4,  38,  18,  39,  40,  19,   4,   8,  41,  42,
          5,  43,   7,   8,  44,  45,   9,  46,  47,  48,  49,  20,  10,
         50,  51,  16,  20,  52,   3,  53,   9,  54,  55,   3,  56,  57,
         58,  59,  60,   2,  61,  21,  62,   4,  63

In [266]:
y = model.predict(x=X, verbose=1)
y[0]

2/2 [==============================] - 0s 19ms/sample


array([2.6908046e-01, 7.2722453e-01, 3.0859001e-03, 6.0913945e-04],
      dtype=float32)

In [267]:
from numpy import argmax
classes = np.argmax(y, axis=1)
classes[0]+1  # adding to zero index


2